In [1]:
import sys
import os

root_path = os.path.abspath('..')
if root_path not in sys.path:
    sys.path.insert(0, root_path)
    
functions_path = os.path.abspath('../Functions')
data_path = os.path.abspath('../Data')
algorithms_path = os.path.abspath('../Algorithms')

sys.path.append(functions_path)
sys.path.append(data_path)
sys.path.append(algorithms_path)

import numpy as np
import pandas as pd
import ruptures as rpt
import matplotlib.pyplot as plt
import time
from Functions.metrics import f_measure, covering

In [2]:
def binseg(ts, n_cps, cost_func="ar", offset=0.05):
    transformer = rpt.Binseg(model=cost_func, min_size=int(ts.shape[0] * offset)).fit(ts)
    return np.array(transformer.predict(n_bkps=n_cps)[:-1], dtype=np.int64)

In [3]:
def evaluate_binseg(dataset, ts, cps, **seg_kwargs):
    try:
        found_cps = binseg(ts, n_cps=len(cps), **seg_kwargs)
        # print("Found CPS:", found_cps)  # Debug print

        f1_score, percision, recall = f_measure({0: cps}, found_cps, margin=int(ts.shape[0] * .01), return_PR=True)
        covering_score = covering({0: cps}, found_cps, ts.shape[0])

        # print("F1 Score:", f1_score)  # Debug print
        # print("Covering Score:", covering_score)  # Debug print

        return dataset, cps, found_cps, np.round(f1_score, 3), np.round(covering_score, 3), percision, recall
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

In [ ]:
# Function to process, plot, and evaluate each PID
def process_plot_evaluate_pid(pid_df, pid, cost_func):
    rwrist_y = pid_df["RWrist_Y"].to_numpy()
    frame_peaks = pid_df["frame_peaks"].to_numpy()
    true_change_points = [i for i, peak in enumerate(frame_peaks) if peak == 1]

    start_time = time.time()
    n_cps = len(true_change_points)
    detected_cps = binseg(rwrist_y, n_cps, cost_func=cost_func)
    end_time = time.time()
    elapsed_time = end_time - start_time

    dataset, true_cps, detected_cps, f1_score, covering_score, precision, recall = evaluate_binseg(pid, rwrist_y, true_change_points, cost_func=cost_func)

    if precision != recall:
        plt.figure(figsize=(15, 6))
        plt.plot(rwrist_y, label=f'Time Series Data for {pid}')
        for cp in detected_cps:
            plt.axvline(x=cp, color='red', linestyle='--', linewidth=1, label='Detected Change Point' if cp == detected_cps[0] else "")
        for cp in true_cps:
            plt.axvline(x=cp, color='green', linestyle='-', linewidth=1, label='True Change Point' if cp == true_cps[0] else "")
        plt.xlabel('Time')
        plt.ylabel('RWrist_Y Value')
        plt.title(f'PID {pid} - Time: {elapsed_time:.4f}s, F1: {f1_score:.6f}, Cover: {covering_score:.6f}, Percision: {precision:.6f}, Recall: {recall:.6f}')
        plt.legend()
        plt.show()

    return dataset, true_cps, detected_cps, f1_score, covering_score, elapsed_time, precision, recall

In [4]:
# Function to process, plot, and evaluate each PID
def process_plot_evaluate_pid(pid_df, pid, cost_func):
    dataset = pid_df["RWrist_Y"].to_numpy()
    frame_peaks = pid_df["frame_peaks"].to_numpy()
    true_cps = [i for i, peak in enumerate(frame_peaks) if peak == 1]

    start_time = time.time()
    n_cps = len(true_cps)
    detected_cps = binseg(dataset, n_cps, cost_func=cost_func)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # dataset, true_cps, detected_cps, f1_score, covering_score, precision, recall = evaluate_binseg(pid, rwrist_y, true_change_points, cost_func=cost_func)
    print(int(dataset.shape[0] * .01))
    detected_cps = binseg(dataset, n_cps, cost_func=cost_func)
    f1_score, precision, recall = f_measure({0: true_cps}, detected_cps, margin=int(dataset.shape[0] * .01), return_PR=True)
    covering_score = covering({0: true_cps}, detected_cps, dataset.shape[0])

    if 1 != 1:
        plt.figure(figsize=(15, 6))
        plt.plot(dataset, label=f'Time Series Data for {pid}')
        for cp in detected_cps:
            plt.axvline(x=cp, color='red', linestyle='--', linewidth=1, label='Detected Change Point' if cp == detected_cps[0] else "")
        for cp in true_cps:
            plt.axvline(x=cp, color='green', linestyle='-', linewidth=1, label='True Change Point' if cp == true_cps[0] else "")
        plt.xlabel('Time')
        plt.ylabel('RWrist_Y Value')
        plt.title(f'PID {pid} - Time: {elapsed_time:.4f}s, F1: {f1_score:.6f}, Cover: {covering_score:.6f}, Percision: {precision:.6f}, Recall: {recall:.6f}')
        plt.legend()
        plt.show()

    return dataset, true_cps, detected_cps, f1_score, covering_score, elapsed_time, precision, recall

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load CSV data
csv_file = '..//Data//Physio//OpenPose-RightWristWithoutSegmentation-full_df.csv'
df = pd.read_csv(csv_file)
cost_func = "ar"

total_time = 0
total_covering = 0
total_f1 = 0
total_percision = 0
total_recall = 0
num_pids = 0

f1_scores = []
percision_scores = []
recall_scores = []
execution_times = []
coverages = []
evaluation_results = []

for pid, group in df.groupby("pid"):
    result = process_plot_evaluate_pid(group, pid, cost_func)
    evaluation_results.append(result)
    total_time += result[-3]
    total_covering += result[-4]
    total_f1 += result[-5]
    total_percision += result[-2]
    total_recall += result[-1]
    num_pids += 1
    execution_times.append(result[-3])
    coverages.append(result[-4])
    f1_scores.append(result[-5])
    percision_scores.append(result[-2])
    recall_scores.append(result[-1])
    
    if pid == "P18_Arch":
        print(f"PID {pid} - True Change Points: {result[1]}, Detected Change Points: {result[2]}")

average_time = total_time / num_pids
average_covering = total_covering / num_pids
average_precision = total_percision / num_pids
average_recall = total_recall / num_pids

average_f1 = 2 * ((average_precision * average_recall) / (average_precision + average_recall))


# Define the directory path for results in the Binseg folder

# Print averages
print(f'Average Time: {average_time:.4f}s, Average Coverage: {average_covering:.6f}, Average F1 Score: {average_f1:.6f}, Average Precision: {average_precision:.6f}, Average Recall: {average_recall:.6f}')

# Save the figure

8
7
7
6
6
7
4
6
8
9
7
8
9
10
7
8
6
8
7
6
12
10
11
9
9
7
7
6
10
9
8
7
11
11
PID P18_Arch - True Change Points: [0, 155, 252, 348, 448, 552, 651, 752, 849, 950, 1042], Detected Change Points: [  85  155  485  550  610  690  755  850  910  985 1045]
11
10
11
12
11
10
9
10
8
7
8
6
6
6
10
11
8
8
7
7
6
8
7
9
8
6
7
5
5
5
6
5
6
5
8
7
7
6
9
9
16
6
8
8
7
6
6
8
5
11
10
7
8
9
9
8
8
8
7
4
3
9
10
10
11
13
13
12
10
7
7
8
7
6
7
5
5
11
12
12
10
7
6
5
6
5
9
9
9
9
8
8
9
8
8
9
7
6
9
8
8
6
6
7
9
6
8
7
7
6
7
8
11
8
9
7
7
5
6
9
9
6
8
8
10
7
7
7
6
7
6
8
9
7
9
8
8
8
10
10
11
11
11
10
10
9
9
7
6
6
6
6
8
6
6
7
6
7
4
8
6
9
5
8
8
7
7
8
10
11
8
Average Time: 0.0519s, Average Coverage: 0.556969, Average F1 Score: 0.319880, Average Precision: 0.312508, Average Recall: 0.327608


In [ ]:
# F1 Scores
plt.figure(figsize=(15, 5))
plt.plot(f1_scores, marker='o', linestyle='-', color='blue', alpha=0.7, label='F1 Score')
plt.title('F1 Scores per PID', fontsize=12)
plt.xlabel('PID Index')
plt.ylabel('F1 Score')
plt.grid(True)
plt.show()

# Execution Times
plt.figure(figsize=(15, 5))
plt.plot(execution_times, marker='o', linestyle='-', color='orange', alpha=0.7, label='Execution Time')
plt.title('Execution Time per PID', fontsize=12)
plt.xlabel('PID Index')
plt.ylabel('Time (seconds)')
plt.grid(True)
plt.show()

# Coverage Scores
plt.figure(figsize=(15, 5))
plt.plot(coverages, marker='o', linestyle='-', color='green', alpha=0.7, label='Coverage')
plt.title('Coverage Score per PID', fontsize=12)
plt.xlabel('PID Index')
plt.ylabel('Coverage')
plt.grid(True)
plt.show()

# Precision Scores
plt.figure(figsize=(15, 5))
plt.plot(percision_scores, marker='o', linestyle='-', color='red', alpha=0.7, label='Precision')
plt.title('Precision Scores per PID', fontsize=12)
plt.xlabel('PID Index')
plt.ylabel('Precision')
plt.grid(True)
plt.show()

# Recall Scores
plt.figure(figsize=(15, 5))
plt.plot(recall_scores, marker='o', linestyle='-', color='purple', alpha=0.7, label='Recall')
plt.title('Recall Scores per PID', fontsize=12)
plt.xlabel('PID Index')
plt.ylabel('Recall')
plt.grid(True)
plt.show()

# Print averages
print(f'Average Time: {average_time:.4f}s, Average Coverage: {average_covering:.6f}, Average F1 Score: {average_f1:.6f}, Average Precision: {average_precision:.6f}, Average Recall: {average_recall:.6f}')